In [68]:
# imports
import requests
import os
import pandas as pd
import time
import numpy as np

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [41]:
# Assigning API key stored in an environtment variable
api_key = os.environ["FOURSQUARE_API_KEY"]

# API endpoint
url = "https://api.foursquare.com/v3/places/search"

# header
headers = {
    "accept": "application/json",
    "Authorization": api_key
}

# Parameters for Foursquare API 
bixi_data = pd.read_csv('bixi_bike_stations.csv') # from city_bikes.ipynb
radius = 1000
category_id = '13032,13065' # id for <Cafe, Coffee, and Tea House> and <Restaurant>
fields = 'name,categories,distance,rating,popularity,price'

# Rate limit in seconds
rate_limit = 1/50

# List used to store data from API request
results_list = []

# Loop through each station and make an API request
for index, station in bixi_data.iterrows():
    params = {
        'll': f"{station['latitude']},{station['longitude']}", # Location: Montreal, QC
        'radius': radius,
        'categories': category_id,
        'fields': fields
    }
    
    response = requests.get(url, params=params, headers=headers)
    # Checking for the status code and error handling
    if response.status_code == 200:
        data = response.json()
        results_list.append(data)
    else:
        print(f"Error: {response.status_code}")
    
    # Rate limit delay to avoid getting restriction on API requests
    time.sleep(rate_limit)

In [ ]:
results_list

##### Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [111]:
# Extracting relevant data

data_list = []
# First loop, since the data from results_list is semi-structured (nested)
for entry in results_list:
    # Second loop to get relevant data inside 'results'
    for result in entry['results']:
        result_data = {
            # Using .get() method in case a key is missing/does not exist
            'POI name': result.get('name'),
            'distance': result.get('distance'),
            'popularity': result.get('popularity'),
            'rating': result.get('rating', np.nan),
            'latitude': repr(entry['context']['geo_bounds']['circle']['center']['latitude']), # Data extracted from the first loop 
            'longitude': repr(entry['context']['geo_bounds']['circle']['center']['longitude']) # Data extracted from the first loop
        }
        data_list.append(result_data)

##### Put your parsed results into a DataFrame

In [116]:
# Set the option to display all rows
pd.set_option('display.max_rows', None)

In [118]:
# If you want to reset the option to the default value (display.max_rows=60)
pd.reset_option('display.max_rows')

In [119]:
df_fs = pd.DataFrame(data_list)

df_fs

,POI name,distance,popularity,rating,latitude,longitude
0,Vua Sandwichs,283,0.957068,8.5,45.516926210319546,-73.56425732374191
1,Mamie Clafoutis,505,0.974074,9.0,45.516926210319546,-73.56425732374191
2,Saint-Houblon,301,0.984992,8.3,45.516926210319546,-73.56425732374191
3,Café Saint-Henri Quartier Latin,294,0.955926,8.2,45.516926210319546,-73.56425732374191
4,Bouillon Bilk,675,0.971504,9.4,45.516926210319546,-73.56425732374191
...,...,...,...,...,...,...
1587,Restaurant Vargas,279,0.987204,8.2,45.50373771539475,-73.56948494911194
1588,Les 3 Brasseurs,195,0.994255,8.0,45.50373771539475,-73.56948494911194
1589,Café Humble Lion,434,0.975087,8.5,45.50373771539475,-73.56948494911194
1590,Reuben's Deli,527,0.990344,8.7,45.50373771539475,-73.56948494911194


In [120]:
# Grouping by 'latitude' and 'longitude' and calculating averages
df_fs_avg = df_fs.groupby(['latitude', 'longitude']).agg({
    'distance': 'mean',
    'popularity': 'mean',
    'rating': 'mean'
}, skipna=True).reset_index()

# Renaming the columns for clarity
df_fs_avg = df_fs_avg.rename(columns={
    'distance': 'average_distance',
    'popularity': 'average_popularity',
    'rating': 'average_rating'
})

df_fs_avg

,latitude,longitude,average_distance,average_popularity,average_rating
0,45.46766565386597,-73.59391784324544,531.2,0.955817,8.550000
1,45.46976497412213,-73.58923405408859,385.5,0.955817,8.550000
2,45.472636238971525,-73.58546018600464,495.1,0.955844,8.330000
3,45.4801160762769,-73.58560770750046,597.7,0.966968,8.690000
4,45.48020822800015,-73.57759863138199,289.6,0.965048,8.870000
...,...,...,...,...,...
155,45.55988367688166,-73.63356828689575,965.0,0.961656,NaN
156,45.56001959585449,-73.534934520776,800.5,0.943463,7.555556
157,45.56157008176612,-73.65338176488876,421.9,0.934679,7.730000
158,45.5616903,-73.610512,404.5,0.924680,6.814286


In [127]:
# Checking why row 155 from df_fs_avg has NaN in the average_rating column

# Filtered by latitude and longitude from df_fs 
filtered_rows = df_fs[(df_fs['latitude'] == '45.55988367688166') & (df_fs['longitude'] == '-73.63356828689575')]
filtered_rows

# Solved: the 2 POIs have no rating

,POI name,distance,popularity,rating,latitude,longitude
60,Café Vienne,935,0.961656,NaN,45.55988367688166,-73.63356828689575
61,L'Oeufrier Cremazie,995,NaN,NaN,45.55988367688166,-73.63356828689575


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating

In [136]:
# Top 10 restaurant by rating from Foursquare

# Drop duplicates based on the 'POI name' column, keeping the first occurrence
unique_restaurants_df = df_fs.drop_duplicates(subset='POI name', keep='first')

# Sort the dataframe by the 'rating' column in descending order
top10_restaurants = unique_restaurants_df.sort_values(by='rating', ascending=False).head(10)

# Resetting index and make it start at 1
top10_restaurants.reset_index(drop=True, inplace=True)
top10_restaurants.index = top10_restaurants.index + 1

top10_restaurants

,POI name,distance,popularity,rating,latitude,longitude
1,Cadet,741,0.981131,9.5,45.516926210319546,-73.56425732374191
2,Bouillon Bilk,675,0.971504,9.4,45.516926210319546,-73.56425732374191
3,Au Kouign-Amann,509,0.968422,9.3,45.52723106564012,-73.58672618865965
4,Olive & Gourmando,528,0.976949,9.3,45.50206,-73.56295
5,Darling,456,0.979904,9.2,45.51689731423111,-73.58910799026489
6,Cafe Italia,277,0.972917,9.2,45.53519006163501,-73.61548215150833
7,Crew Collective & Café,294,0.989859,9.2,45.50206,-73.56295
8,Milos Restaurant,698,0.964960,9.2,45.527371996141,-73.60398352146149
9,Damas,811,0.978833,9.2,45.52357479400008,-73.62344294786453
10,Maison Boulud,410,0.961827,9.2,45.49658063218649,-73.5762146115303
